## Quantum Phase Estimation

In [6]:
import qiskit
from qiskit import QuantumCircuit, Aer, transpile
from qiskit.visualization import plot_histogram
from qiskit.providers.aer import AerSimulator

In [10]:
# Function to setup QPE circuit for a given eigenvector
def qpe_for_z(eigenvector):
    qc = QuantumCircuit(2, 1)  # 2 qubits, 1 classical bit for measurement

    # Prepare the eigenvector on the second qubit
    if eigenvector == '1':
        qc.x(1)

    # Apply Hadamard to the ancilla
    qc.h(0)

    # Controlled-Z operation
    qc.cz(0, 1)

    # Inverse QFT (Hadamard for single qubit)
    qc.h(0)

    # Measurement
    qc.measure(0, 0)

    return qc


In [12]:

# Eigenvectors '0' and '1'
eigenvectors = ['0', '1']
results = {}

simulator = AerSimulator()

for vec in eigenvectors:
    qc = qpe_for_z(vec)
    compiled_circuit = transpile(qc, simulator)
    result = simulator.run(compiled_circuit, shots=1000).result()
    counts = result.get_counts()
    results[f"Eigenvector |{vec}⟩"] = counts


In [19]:
results

{'Eigenvector |0⟩': {'0': 1000}, 'Eigenvector |1⟩': {'1': 1000}}

In [21]:

# Display the results
for key, val in results.items():
    print(key)
    plot_histogram(val, title=key).savefig(f"qpe_{key}.png")


Eigenvector |0⟩
Eigenvector |1⟩


C:\Users\techi\AppData\Local\Temp\ipykernel_5392\3592167987.py:4: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plot_histogram(val, title=key).show()
